In [12]:
from src.embeddings import VietnameseEmbeddings
import uuid
from langchain.retrievers import EnsembleRetriever
from langchain.storage import InMemoryByteStore
from langchain_community.vectorstores import DeepLake
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import MultiVectorRetriever
from langchain_community.retrievers import BM25Retriever


In [2]:
model_name="dangvantuan/vietnamese-embedding"
model_kwargs={'device': 'cpu'}

embedding = VietnameseEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

/home/ausar/miniconda3/envs/rag_agent/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
loader = DirectoryLoader('data', glob="**/*.txt", loader_cls=TextLoader, show_progress=True)
docs = loader.load()


100%|██████████| 141/141 [00:00<00:00, 3589.86it/s]


In [13]:
# The vectorstore to use to index the child chunks
vectorstore = DeepLake(embedding=embedding)
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
bm25_retriever = BM25Retriever.from_documents(documents=docs)
doc_ids = [str(uuid.uuid4()) for _ in docs]

Deep Lake Dataset in ./deeplake/ already exists, loading from the storage


In [14]:
# The splitter to use to create smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

In [15]:
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

In [16]:
retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))


Creating 349 embeddings in 1 batches of size 349::   0%|          | 0/1 [00:00<?, ?it/s]
Creating 349 embeddings in 1 batches of size 349:: 100%|██████████| 1/1 [00:22<00:00, 22.19s/it]

Dataset(path='./deeplake/', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (349, 768)  float32   None   
    id        text      (349, 1)     str     None   
 metadata     json      (349, 1)     str     None   
   text       text      (349, 1)     str     None   


In [17]:
retriever.vectorstore.similarity_search("Điểm M")[0]

Document(page_content='Điểm “Miễn” (Điểm M): Một môn học A được ghi nhận “điểm miễn” (gọi là “điểm M”) cho các trường hợp sau: * **Trường hợp 1:** Sinh viên đã hoàn thành môn học tương đương tại Trường, nộp đơn đề nghị xét công nhận kết quả học tập, nếu sinh viên được xét công nhận điểm miễn sẽ được ghi điểm M hoặc ghi nhận điểm theo kết quả môn tương đương đã đạt được. * **Trường hợp 2:** Sinh viên đã hoàn thành', metadata={'source': 'data/ts2021-tro-ve-sau/Chương III. ĐÁNH GIÁ KẾT QUẢ HỌC TẬP/Điều 25 Các điểm đặc biệt trong bảng điểm/Điểm “Miễn” (Điểm M).txt', 'doc_id': '9b479ce4-9476-4044-b28e-f97494157342'})

In [18]:
bm25_retriever.k = 5

In [19]:
# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5]
)

In [20]:
docs = ensemble_retriever.invoke("Điểm M")

In [21]:
docs

[Document(page_content='Chương III. ĐÁNH GIÁ KẾT QUẢ HỌC TẬP\nĐiều 25. Các điểm đặc biệt trong bảng điểm\nĐiểm “Miễn” (Điểm M): Một môn học A được ghi nhận “điểm miễn” (gọi là “điểm M”) cho các trường hợp sau: * **Trường hợp 1:** Sinh viên đã hoàn thành môn học tương đương tại Trường, nộp đơn đề nghị xét công nhận kết quả học tập, nếu sinh viên được xét công nhận điểm miễn sẽ được ghi điểm M hoặc ghi nhận điểm theo kết quả môn tương đương đã đạt được. * **Trường hợp 2:** Sinh viên đã hoàn thành chương trình học tại Trường đại học khác, sinh viên thuộc diện chuyển trường hoặc chuyển từ nước ngoài về sẽ được xem xét chuyển điểm các môn học theo quy định tại Điều 27 quy chế này. Điểm môn học được công nhận miễn sẽ được ghi điểm M hoặc ghi nhận điểm theo kết quả môn tương đương đã đạt được. * **Trường hợp 3:** Khi sinh viên đã hoàn tất một số môn học đặc thù (như Tiếng Anh, Cơ sở tin học, Kỹ năng,...) hoặc môn có điểm môn học là kết quả thi chứng chỉ đầu ra và đạt yêu